# Tratamiento SAC

In [1]:
import pandas as pd
from data import CWTR_SUM
from helpers.bases import tabla
import numpy as np

### Cálculo del SAC correspondiente a cada semestre

In [2]:
#filtro el df que tiene todos los conceptos y dejo solamente el del SAC
df_sac = CWTR_SUM.query('CCn == "/S02"')

# creo df con el filtro correspondiente a cada semestre
df_1erSem = df_sac.query('1 <= mes <= 6')
df_2doSem = df_sac.query('7 <= mes <= 12')

#como tengo varios sac por mes, los agrupo para que me quede uno por cada semestre
df_1erSem = df_1erSem.groupby(['legajo','CCn'])['importe'].sum().reset_index() 
df_2doSem = df_2doSem.groupby(['legajo','CCn'])['importe'].sum().reset_index() 

In [3]:
#me traigo las remuneraciones para determinar si el SAC queda gravado o exento (sin ceros)
bases_tramos = tabla("bases_tramos").query('importe != 0')

# #creo un df por cada semestre y agrupo segun el legajo, calculando el promedio
filtro_1erSem = bases_tramos.query('1 <= mes <= 6')
filtro_1erSem = filtro_1erSem.groupby(['legajo'])['importe'].mean().reset_index() 
filtro_2doSem = bases_tramos.query('7 <= mes <= 12')
filtro_2doSem = filtro_2doSem.groupby(['legajo'])['importe'].mean().reset_index()

TOPE_1_SEMESTRE = 280_792
TOPE_2_SEMESTRE = 330_000

# #renombro las columnas para poder luego unir los df segun el legajo y determino si esta gravado o exento 
SAC_1erSem =  filtro_1erSem.rename(columns={'importe':'Prom_SAC_1erSem'})
SAC_1erSem["Sit_1er_Sem"] = np.where(SAC_1erSem["Prom_SAC_1erSem"] >= TOPE_1_SEMESTRE, "Gravado", "Exento")
SAC_2doSem =  filtro_2doSem.rename(columns={'importe':'Prom_SAC_2doSem'})
SAC_2doSem["Sit_2do_Sem"] = np.where(SAC_2doSem["Prom_SAC_2doSem"] >= TOPE_2_SEMESTRE, "Gravado", "Exento")

# #hago la union de los dos df con un merge y me traigo los datos de ambas tablas (outer)
df_promedios_SAC = pd.merge(SAC_1erSem, SAC_2doSem, on="legajo", how="outer")
df_promedios_SAC.to_clipboard()

In [4]:
#guardo el archivo junto con el resto de los parquet para luego llamarlo en otro lado 
df_promedios_SAC.to_parquet('../bases-ganancias-2022/middle/df_promedios_SAC.parquet', compression='brotli')